# Conservation Analysis and Epitope Prediction


#### Author: C. Mazzaferro, K. Fisch
#### Email: cmazzafe@ucsd.edu
#### Date: August 2016
 
## Outline of Notebook
<a id = "toc"></a>
1. <a href = "#background">Background</a>
2. <a href = "#ConsA">Conservation Analysis</a>
    * <a href = "#BLAST">BLAST-P</a>
    * <a href = "#MSA">Multiple Sequence Alignment</a>
    * <a href = "#Cons">Conservation Score Prediction</a>
3. <a href = "#ep_pred">Windowing and New Epitope Prediction</a>
    * <a href = "#clustering">Epitope Clustering Analysis</a>

This workflow aims at finding peptide substitute candidates by swapping 1 AA at a time from a prioritized list of peptides. This prioritized list was found using the workflow - Base Workflow.

## Finding similar peptides by doing simple AA swaps
The peptides will be written to a fasta file which will be sent to netMHCcons. Results will be analyzed subsequentially


In [12]:
import pandas

csv_path = '/Users/carlomazzaferro/Desktop/Test_Workflow_From_Scratch/all_high_aff_aas.csv'
high_aa_df = pandas.read_csv(csv_path)

In [13]:
from nepitope import pep_utils
fasta_files_dir = '/Users/carlomazzaferro/Desktop/Test_Workflow_From_Scratch/swaps/'
pep_utils.find_swaps_write_to_fasta(high_aa_df, fasta_files_dir)

In [15]:
import importlib
from nepitope import net_MHC_interface
importlib.reload(net_MHC_interface)



fasta_file = '/Users/carlomazzaferro/Desktop/PROJECT_LOC_TEST/prots_in_fasta.fasta'
net_mhc_path = '/Users/carlomazzaferro/Desktop/BINF_Tools/netMHC-4.0/netMHC'


In [34]:
import os

fasta_files = '/Users/carlomazzaferro/Desktop/Test_Workflow_From_Scratch/swaps/StreptococcusPyogenes_reference/'
fls = glob.glob(fasta_files + '*')
net_mhc_comm = []

for i in fls:
    base_name = os.path.basename(i).split('.')[0]
    dat = base_name.split('_')
    allele = dat[-3]
    nmer = int(dat[-1])        
    file_name_out =  base_name + '.csv'
    net_mhc_comm.append('%s -a %s -f %s -l %i -xls -xlsfile %s' % (net_mhc_path, allele, i,
                                                                              nmer, file_name_out))

In [40]:
net_mhc_cmd_str = fasta_files + 'Results/net_mhc_command.txt'
with open(net_mhc_cmd_str, 'w') as cmd:
    for i in net_mhc_comm:
        cmd.write(i + '\n')

### Load Results From netMHC
The swaps are sent to netMHCCons, and the results are aggragated under the class FileConsolidation in the module scoring_utils

In [4]:
from nepitope import mhc_utils, summary_data
import glob
nmers = [8, 9,10,11]

#files saved from netMHCcons as split_*i*_prediction_*n*_mer for each split/nmer
filepath = '/Users/carlomazzaferro/Desktop/Test_IEDB/newer_pipeline_files/consistent_aws_netMHC40/swaps/mhc_preds_per_swap/'
fasta_file = '/Users/carlomazzaferro/Desktop/Test_IEDB/newer_pipeline_files/fasta_from_study_dealigned.fasta'

file_pattern = '_swap_*'

aggregate_all = mhc_utils.FileConsolidation(filepath, fasta_file)

files = glob.glob(filepath + file_pattern)
original_peps_and_pos = []

for file in files:
    split_ = file.split('/')[-1]
    split_2 = split_.split('_')
    original_peps_and_pos.append([split_2[2], split_2[4]])

### Return them in a list of dataframes
summary_data is the module that performs all the data aggregation on newly found peptides.

In [5]:
list_results = aggregate_all.return_df_list()

In [8]:
summ_data = summary_data.SummaryData()
summ_data_analysis = summ_data.summarize_data_for_each_mhc_pred(list_results, original_peps_and_pos, high_aa_df)

In [9]:
pd = pandas.concat(summ_data_analysis.container)
cl = pd.drop_duplicates()
#cl.to_csv('/Users/carlomazzaferro/Desktop/all_preds_swaps.csv')

### Return summary data in a dataframe
Other methods and attributes are present as well

In [41]:
summ_data_analysis.summary_df['original peptide'].values.tolist()[0:20]

['AENIIHLFTL',
 'AERGGLSEL',
 'AHDAYLNAV',
 'AHMIKFRGHF',
 'AILLSDILRV',
 'AILSARLSK',
 'ALSLGLTPNF',
 'AYHEKYPTI',
 'DEIIEQISEF',
 'EEVVKKMKNY',
 'EFVYGDYKVY',
 'EIIEQISEF',
 'EIIEQISEFS',
 'ESEFVYGDY',
 'EVVKKMKNY',
 'FANRNFMQL',
 'FLYLASHYEK',
 'FVYGDYKVY',
 'FYSNIMNFF',
 'GELQKGNEL']

In [11]:
summ_data_analysis.summary_df.sort_values(by='original pos')

,nmer,allele,num high affinity peps,num med affinity peps,num low affinity peps,num no affinity peps,original peptide,original pos,top scoring peptides
21,9,HLA-A0201,74,64,14,28,GLDIGTNSV,7,"[GLDIGTNSD, GHDIGTNSV, GRDIGTNSV, GDDIGTNSV, G..."
89,9,HLA-B0801,57,80,38,5,VPSKKFKVL,26,"[VPSKKFKVK, VPSKKFKVR, VPSKKFKVD, VPSKKFKVN, V..."
88,9,HLA-B0702,94,50,11,25,VPSKKFKVL,26,"[VDSKKFKVL, VCSKKFKVL, VWSKKFKVL, VESKKFKVL, V..."
20,9,HLA-B4001,129,15,15,21,GETAEATRL,55,"[GGTAEATRL, GPTAEATRL, GITAEATRL, GLTAEATRL, G..."
74,9,HLA-B2705,88,61,16,15,RRYTRRKNR,69,"[RDYTRRKNR, RPYTRRKNR, RCYTRRKNR, REYTRRKNR, R..."
75,10,HLA-B2705,145,34,6,15,RRYTRRKNRI,69,"[RPYTRRKNRI, RDYTRRKNRI, RFYTRRKNRI, RGYTRRKNR..."
65,9,HLA-B2705,70,76,18,15,RRKNRICYL,73,"[RDKNRICYL, RPKNRICYL, REKNRICYL, RGKNRICYL, R..."
7,9,HLA-A2402,83,62,15,20,AYHEKYPTI,126,"[APHEKYPTI, AEHEKYPTI, AKHEKYPTI, ADHEKYPTI, A..."
64,10,HLA-B1501,144,41,9,6,RLIYLALAHM,151,"[RLIYLALAHR, RLIYLALAHK, RLIYLALAHD, RLIYLALAH..."
31,9,HLA-A2402,121,28,25,6,IYLALAHMI,153,"[IPLALAHMI, IELALAHMI, IDLALAHMI, IKLALAHMI, I..."


### Write files out

In [58]:
fasta_dir = '/Users/carlomazzaferro/Desktop/fasta_swaps_top_20_percent.fasta'
summ_data_analysis.write_peptides_to_fasta(fasta_dir)

In [59]:
csv_path = '/Users/carlomazzaferro/Desktop/Test_IEDB/newer_pipeline_files/consistent_aws_netMHC40/summary_results_per_prediction.csv'
summ_data_analysis.summary_df.sort_values(by='original pos').to_csv(path_or_buf=csv_path, sep=',')

In [62]:
basepath = '/Users/carlomazzaferro/Desktop/Test_IEDB/newer_pipeline_files/consistent_aws_netMHC40/csv_for_each_pred/'
for idx, val in enumerate(list_results):
    original_pep = original_peps_and_pos[idx][0]
    pos = original_peps_and_pos[idx][1]
    allele = val.Allele.unique()[0]
    nmer = val['n-mer'].unique()[0]
    name = "_".join([original_pep, pos, allele, str(nmer)])
    val['Peptide'] = val['Peptide'].str.replace('X', '-')
    write_to_csv(val, basepath + name + '.csv')
    

In [61]:
def write_to_csv(df, path_name):
    df.to_csv(path_name, sep=',')
    